<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Valores-nulos" data-toc-modified-id="Valores-nulos-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Valores nulos</a></span></li><li><span><a href="#Visualización" data-toc-modified-id="Visualización-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Visualización</a></span></li></ul></li><li><span><a href="#Pre-procesamiento" data-toc-modified-id="Pre-procesamiento-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-procesamiento</a></span><ul class="toc-item"><li><span><a href="#Tratamientos-valores-duplicados" data-toc-modified-id="Tratamientos-valores-duplicados-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Tratamientos valores duplicados</a></span></li><li><span><a href="#Completar-gaps-de-la-serie" data-toc-modified-id="Completar-gaps-de-la-serie-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Completar gaps de la serie</a></span></li><li><span><a href="#Información-temporal" data-toc-modified-id="Información-temporal-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Información temporal</a></span></li><li><span><a href="#Estimar-valor-gaps" data-toc-modified-id="Estimar-valor-gaps-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Estimar valor gaps</a></span></li><li><span><a href="#Visualización" data-toc-modified-id="Visualización-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Visualización</a></span></li></ul></li><li><span><a href="#Descomposicion-de-Series-Temporales" data-toc-modified-id="Descomposicion-de-Series-Temporales-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Descomposicion de Series Temporales</a></span></li><li><span><a href="#Autocorrelación-y-autocorrelación-parcial" data-toc-modified-id="Autocorrelación-y-autocorrelación-parcial-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Autocorrelación y autocorrelación parcial</a></span></li><li><span><a href="#Bibliografía" data-toc-modified-id="Bibliografía-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Bibliografía</a></span></li></ul></div>

## Librerías

In [1]:
# Librerías
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from plotly.offline import init_notebook_mode, iplot
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Paquetes
from modules import analisis

# Configuración de librerías y paquetes
%matplotlib inline
%load_ext autoreload
%autoreload 2
init_notebook_mode(connected = True)

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Setup

In [2]:
df = pd.read_csv('df_energy_price_spain.csv', sep=',',encoding='utf-8-sig')
df.drop('Unnamed: 0', axis=1,inplace=True)
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df.sort_values('date', ascending=True, inplace=True)

In [3]:
df.tail()

,date,value
35634,2020-12-31 20:00:00,60.54
35635,2020-12-31 21:00:00,56.75
35636,2020-12-31 22:00:00,52.44
35637,2020-12-31 23:00:00,51.86
35638,2021-01-01 00:00:00,52.26


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
value,35639.0,46.397866,14.890721,0.01,37.58,47.02,56.01,101.99


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35639 entries, 0 to 35638
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    35639 non-null  datetime64[ns]
 1   value   35639 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 835.3 KB


In [6]:
# Tamaño del dataset
df.shape

(35639, 2)

### Valores nulos

In [7]:
df['value'].isna().any()

False

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35639 entries, 0 to 35638
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    35639 non-null  datetime64[ns]
 1   value   35639 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 835.3 KB


### Visualización

In [9]:
analisis.plot_precio_energia_temporal(df)

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

In [13]:
(df['date'].value_counts()!=1).sum()

27

## Pre-procesamiento

### Tratamientos valores duplicados

In [10]:
def encontrar_duplicados(df):
    """
    Encontrar duplicados a partir de la marca temporal de la muestra (columna
    date del dataframe)
    
    Args:
        df {[dataframe]}: DataFrame con columnas "date" y "value"
    
    Returns:
        num_duplicated {[int]}: Número de elementos duplicados
        df_duplicated {[dataframe]}: DataFrame de duplicados
    """
    # Obtener número total de duplicados
    num_duplicated = df['date'].duplicated().sum()
    
    # Encontrar duplicados del dataframe
    duplicated_rows = df.loc[df['date'].duplicated()]['date']
    
    df_duplicated = df.loc[df['date'].isin(duplicated_rows)]
    
    return num_duplicated, df_duplicated

In [11]:
num_duplicated, df_duplicated = encontrar_duplicados(df)
print(f'Numéro duplicados: {num_duplicated}')

Numéro duplicados: 27


In [12]:
df_duplicated

,date,value
13030,2017-10-30 01:00:00,44.00
13031,2017-10-30 01:00:00,48.37
17951,2018-07-04 01:00:00,61.40
17950,2018-07-04 01:00:00,61.40
17952,2018-07-04 02:00:00,59.19
17953,2018-07-04 02:00:00,59.19
17954,2018-07-04 03:00:00,59.09
17955,2018-07-04 03:00:00,59.09
17956,2018-07-04 04:00:00,59.19
17957,2018-07-04 04:00:00,59.19


Existen pequeñas variaciones en ciertas horas que están duplicadas, nos quedamos con la media de estos valores

In [21]:
def tratar_duplicados(df):
    """
    Tratar duplicados a partir de la marca temporal de la muestra (columna
    date del dataframe) y calcular la media de los valores repetidos de la serie.
    
    Args:
        df {[dataframe]}: DataFrame con columnas "date" y "value"
    
    Returns:
        num_duplicated {[int]}: Número de elementos duplicados
        df_duplicated {[dataframe]}: DataFrame de duplicados
    """
    # Encontrar duplicados del dataframe
    duplicated_rows = df.loc[df['date'].duplicated()]['date']
    df_duplicated = df.loc[df['date'].isin(duplicated_rows)]
    
    # Variable de almacenamiento de resultados
    resultados = list()
    
    for marca_temporal in df_duplicated.date.unique():
        valores_duplicados = df_duplicated.loc[df_duplicated['date'] == marca_temporal]
        series_result = pd.Series(
            {
                'date':marca_temporal, 
                'value':valores_duplicados['value'].mean()
            }
        )
        resultados.append(series_result)
        
    resultados = pd.concat(resultados, axis=1).T
    
    
    # Añadir resultados
    df_ = df.set_index('date').drop(resultados['date'], axis=0).reset_index()
    
    df_result = pd.concat([df_,resultados], axis=0)
    
    return df_result

In [22]:
df = tratar_duplicados(df)

In [23]:
num_duplicated, df_duplicated = encontrar_duplicados(df)
print(f'Numéro duplicados: {num_duplicated}')

Numéro duplicados: 0


In [24]:
df.head()

,date,value
0,2016-01-01 01:00:00,48.55
1,2016-01-01 02:00:00,40.0
2,2016-01-01 03:00:00,33.1
3,2016-01-01 04:00:00,28.11
4,2016-01-01 05:00:00,27.13


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35612 entries, 0 to 26
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    35612 non-null  datetime64[ns]
 1   value   35612 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 834.7+ KB


### Completar gaps de la serie

In [26]:
def comprobar_fechas(df):
    """
    Mostrar por pantalla fecha de inicio y fin de los datos y los 
    gaps.
    
    Args:
    --------
        df: DataFrame con columna 'date' en formato datetime
    """
    first_date = df['date'].min()
    last_date = df['date'].max()
    samples = df.shape[0]
    
    print('Data series')
    print(f'Samples: {samples}')
    print(f'From "{first_date}" to "{last_date}"')
    print(5*'-', end=2*'\n')
    
    # Toma la diferencia desde la primera fila
    deltas = df['date'].diff()[1:]

    # Filtar diffs Filter diffs (hour > 1)
    gaps = deltas[deltas > timedelta(hours=1)]

    # Print results
    print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}')
    for i, g in gaps.iteritems():
        gap_start = df.iloc[i-1]['date']
        print(f'Start: {datetime.strftime(gap_start, "%Y-%m-%d %H:%M:%S")} | '
              f'Duration: {str(g.to_pytimedelta())}')

In [27]:
def completar_gaps(df, start_date, end_date, freq='1H'):
    """
    Completar las faltas de horas del dataframe con valor 0 con la finalidad
    de poder analizar correctamente la serie temporal.
    
    Args:
        df: DataFrame de análisis con columnas "date" y "value"
        
    Returns:
        df_result: DataFrame resultado
    """
    # Crear dataframe sin gaps todo con ceros
    df_nogaps = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq=freq), columns=['date'])
    df_nogaps['value'] = np.nan
    
    # Concatenar dataframe nuevo con el original para completar los gaps
    df_result = pd.concat([df, df_nogaps.loc[~df_nogaps['date'].isin(df['date'])]], axis=0)
    
    return df_result.sort_values('date', ascending=True).reset_index(drop=True)

In [28]:
# Comprobar columna datetime
comprobar_fechas(df)

Data series
Samples: 35612
From "2016-01-01 01:00:00" to "2021-01-01 00:00:00"
-----

303 gaps with average gap duration: 4 days 18:40:47.524752475
Start: 2016-01-19 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-01-21 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-01-24 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-01-27 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-01-30 00:00:00 | Duration: 3 days, 1:00:00
Start: 2016-02-13 00:00:00 | Duration: 2 days, 1:00:00
Start: 2016-02-25 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-02-29 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-03-03 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-03-07 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-03-14 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-03-27 23:00:00 | Duration: 2:00:00
Start: 2016-03-30 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-04-05 00:00:00 | Duration: 2 days, 1:00:00
Start: 2016-04-08 00:00:00 | Duration: 1 day, 1:00:00
Start: 2016-04-10 00:00:00 | Duration: 1 day, 

In [30]:
df_nogaps = completar_gaps(df, "2016-01-01 01:00:00", "2021-01-01 00:00:00")

# Comprobar columna datetime
comprobar_fechas(df_nogaps)

# Establecer frequencia al datetime index
df_nogaps.set_index('date', inplace=True)
df_nogaps.index.freq = '1H'
df_nogaps.reset_index(inplace=True)

Data series
Samples: 43848
From "2016-01-01 01:00:00" to "2021-01-01 00:00:00"
-----

0 gaps with average gap duration: NaT


### Información temporal

In [ ]:
def add_temporal_info(df):
    """
    Añadir información temporal.
    """
    df['quarter'] = df.date.dt.quarter
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['dayofweek'] = df.date.dt.dayofweek
    df['hour'] = df.date.dt.hour
    
    return df

In [ ]:
df_bis = add_temporal_info(df_nogaps)

In [ ]:
df_bis.sample(10)

### Estimar valor gaps

In [ ]:
def calcular_valor_gaps(df):
    """
    Versión 1.
    
    Método de cálculo del valor de las fechas introducidas manualmente para
    completar la serie temporal.
    
    Procedimiento:
        (1) Obtener muestras cuyo valor es 0 (introducidos manualmente por
            la función "completar_gaps")
        (2) Para cada muestra, se calcula la mediana del valor de las muestras
            con la misma hora.
        (3) Almacenamiento de la mediana en el valor introducido
        
    Args:
        df {[dataframe]}: DataFrame con columnas "date" y "value"
        
    Returns:
        df_final {[dataframe]}: DataFrame resultado
    """

    df_result = df.copy()
    
    # Bucle iterativo con valor 0 (introducido para rellenar gaps)
    for row in df.loc[df['value'].isna()].iterrows():
        # Seleccionar muestra
        gap_series = row[1]

        # Encontrar muestras similares
        samples_similares = df.loc[(df['hour']  == gap_series['hour']) & 
                                   (df['month'] == gap_series['month']) &
                                   (df['year'] == gap_series['year']) &
                                   (df['value'] != np.nan)]
        
        # Calcular precio medio de energía
        new_value = samples_similares['value'].mean()

        # Almacenamiento 
        df_result.loc[df_result['date']==gap_series['date'],'value'] = new_value
        
    return df_result

In [ ]:
df_final = calcular_valor_gaps(df_bis)

In [ ]:
df_final.sample(10)

### Visualización

In [ ]:
def plot_line(df):
    """
    Visualización de serie temporal
    """
    fig = px.line(df, 
              x ='date',
              y ='value',
              template = 'plotly_white'   )
    fig.update_layout(title = '<b>ElectroStock S.A⚡</b> Precio (€) de la energía diario (promedio del día) en el mercado eléctrico español.')
    fig.show()

In [ ]:
plot_line(df_final)

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Descomposicion de Series Temporales

In [ ]:
# Crear dataframe de trabajo
df_seas_decompose = df_final.set_index('date')

# Calcular descomposición serie temporal
result = seasonal_decompose(df_seas_decompose['value'], model='additive')

# Almacenamiento de resultados
df_seas_decompose['observed'] = result.observed
df_seas_decompose['trend'] = result.trend
df_seas_decompose['seasonal'] = result.seasonal
df_seas_decompose['resid'] = result.resid

# Resetear indice
df_seas_decompose.reset_index(inplace=True)

In [ ]:
fig = make_subplots(rows=4, cols=1)

fig.add_trace(
    go.Scatter(
        x=df_seas_decompose['date'], 
        y=df_seas_decompose['observed'],
        name="observed"),
              row=1, col=1)

fig.add_trace(
    go.Scatter(
        x=df_seas_decompose['date'], 
        y=df_seas_decompose['trend'],
        name="trend"),
              row=2, col=1)

fig.add_trace(
    go.Scatter(
        x=df_seas_decompose['date'], 
        y=df_seas_decompose['seasonal'],
        name="seasonal"),
              row=3, col=1)

fig.add_trace(
    go.Scatter(
        x=df_seas_decompose['date'], 
        y=df_seas_decompose['resid'],
        name="resid"),
              row=4, col=1)

fig.update_layout(title='Descomposición serie temporal')

fig.show()

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

## Autocorrelación y autocorrelación parcial

In [ ]:
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
plot_pacf(df_final['value'], lags = 30)
plt.show()

In [ ]:
plot_acf(df_final['value'], lags = 30)
plt.show()

## Bibliografía

- [1] Series temporales [url: https://nwfsc-timeseries.github.io/atsa-labs/sec-tslab-decomposition-of-time-series.html]